In [1]:
import pandas as pd
import spacy
from sklearn import model_selection
import numpy as np


from tensorflow import keras
from tensorflow.keras import layers, models

In [2]:
df = pd.read_json(open("../../data/data.json", "r", encoding="utf8"))


In [3]:
texts_len = df['text'].apply(len)
df.drop(df[texts_len<50].index, inplace=True)

In [4]:
max_features = 10000  # maximum number of words in vocabulari 5000
max_len = 150  # max length of string

In [5]:
joined_text = df['title'] + df['text']
X = keras.preprocessing.sequence.pad_sequences(list(joined_text), maxlen=max_len, padding='post')
Y = np.array(df['themes'].to_list())

In [6]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.1, random_state=42)

### Model 1 simple Dense Layers

In [7]:
embedding_dim = 64

model1 = keras.models.Sequential([
  keras.layers.Embedding(input_dim=max_features,
                           output_dim=embedding_dim,
                           input_length=max_len),
  keras.layers.Flatten(),
  keras.layers.Dense(2000, activation='relu'),
  keras.layers.Dropout(0.3),
  keras.layers.Dense(500, activation='relu'),
  keras.layers.Dropout(0.3),
  keras.layers.Dense(100, activation='relu'),
  keras.layers.Dense(len(Y[0]), activation='sigmoid')
])

model1.compile(optimizer='nadam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 64)           640000    
_________________________________________________________________
flatten (Flatten)            (None, 9600)              0         
_________________________________________________________________
dense (Dense)                (None, 2000)              19202000  
_________________________________________________________________
dropout (Dropout)            (None, 2000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               1000500   
_________________________________________________________________
dropout_1 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               5

In [8]:
epochs = 5
model1.fit(np.array(X_train), np.array(Y_train),
          #batch_size=128,
          validation_data=(np.array(X_test),np.array(Y_test)),
          epochs=epochs)

Epoch 1/5
367/367 [==============================] - 84s 230ms/step - loss: 19.6905 - categorical_accuracy: 0.0067 - val_loss: 18.7863 - val_categorical_accuracy: 0.0061
Epoch 2/5
367/367 [==============================] - 86s 235ms/step - loss: 18.5352 - categorical_accuracy: 0.0099 - val_loss: 18.6988 - val_categorical_accuracy: 0.0069
Epoch 3/5
367/367 [==============================] - 82s 225ms/step - loss: 16.2386 - categorical_accuracy: 0.0558 - val_loss: 19.2203 - val_categorical_accuracy: 0.0537
Epoch 4/5
367/367 [==============================] - 83s 226ms/step - loss: 13.7125 - categorical_accuracy: 0.1889 - val_loss: 20.6579 - val_categorical_accuracy: 0.0668
Epoch 5/5
367/367 [==============================] - 88s 240ms/step - loss: 12.2265 - categorical_accuracy: 0.2883 - val_loss: 21.8934 - val_categorical_accuracy: 0.0583


In [9]:
score = model1.evaluate(np.array(X_test), np.array(Y_test)) 

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

41/41 [==============================] - 1s 14ms/step - loss: 21.8934 - categorical_accuracy: 0.0583
Test Score: 21.893369674682617
Test Accuracy: 0.05832693725824356


In [10]:
'''
class_names = ['out of cat ', 'in cat']
cm = confusion_matrix(Y_test, np.rint(y_pred))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot()
plt.show()'''

NameError: name 'confusion_matrix' is not defined

### Model 2 LSTM

In [12]:
embedding_dim =100 

model2 = keras.Sequential([
    layers.Embedding(max_features, embedding_dim, input_length=max_len),
    layers.SpatialDropout1D(0.2),
    layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    layers.Dense(len(Y[0]), activation='sigmoid')
])

In [13]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [14]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 100)          1000000   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 150, 100)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense_4 (Dense)              (None, 112)               11312     
Total params: 1,091,712
Trainable params: 1,091,712
Non-trainable params: 0
_________________________________________________________________


In [15]:
max_features = 10000  # maximum number of words in vocabulari 5000
max_len = 150  # max length of string
batch_size = 128
epochs = 5

early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)
history = model2.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[early_stopping_callback])

Epoch 1/5
83/83 [==============================] - 33s 401ms/step - loss: 20.0450 - categorical_accuracy: 0.0066 - val_loss: 20.1598 - val_categorical_accuracy: 0.0094
Epoch 2/5
83/83 [==============================] - 32s 388ms/step - loss: 19.4389 - categorical_accuracy: 0.0064 - val_loss: 20.1142 - val_categorical_accuracy: 0.0094
Epoch 3/5
83/83 [==============================] - 29s 346ms/step - loss: 19.4231 - categorical_accuracy: 0.0064 - val_loss: 20.0981 - val_categorical_accuracy: 0.0094
Epoch 4/5
83/83 [==============================] - 31s 373ms/step - loss: 19.4071 - categorical_accuracy: 0.0064 - val_loss: 20.0890 - val_categorical_accuracy: 0.0094
Epoch 5/5
83/83 [==============================] - 32s 381ms/step - loss: 19.3945 - categorical_accuracy: 0.0064 - val_loss: 20.0769 - val_categorical_accuracy: 0.0094


In [16]:
accr = model2.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

41/41 [==============================] - 1s 36ms/step - loss: 18.8608 - categorical_accuracy: 0.0061
Test set
  Loss: 18.861
  Accuracy: 0.006


In [17]:
output_dim = 100

inputs = keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, output_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

predictions = layers.Dense(len(Y[0]), activation='softmax', name="predictions")(x)

model3 = keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model3.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
model3.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 100)         1000000   
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 100)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 128)         89728     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         114816    
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512 

In [19]:
batch_size = 64
epochs = 5
model3.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Epoch 1/5
165/165 [==============================] - 8s 48ms/step - loss: 632743.8125 - accuracy: 0.0207 - val_loss: 3133641.7500 - val_accuracy: 0.0094
Epoch 2/5
165/165 [==============================] - 12s 72ms/step - loss: 62354940.0000 - accuracy: 0.0209 - val_loss: 181421184.0000 - val_accuracy: 0.0094
Epoch 3/5
165/165 [==============================] - 11s 66ms/step - loss: 747068416.0000 - accuracy: 0.0200 - val_loss: 1590319872.0000 - val_accuracy: 0.0094
Epoch 4/5
165/165 [==============================] - 9s 56ms/step - loss: 3776402688.0000 - accuracy: 0.0216 - val_loss: 6591901696.0000 - val_accuracy: 0.0537
Epoch 5/5
165/165 [==============================] - 8s 46ms/step - loss: 12139752448.0000 - accuracy: 0.0199 - val_loss: 18524616704.0000 - val_accuracy: 0.0179


In [20]:
accr = model3.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

41/41 [==============================] - 1s 12ms/step - loss: 18964541440.0000 - accuracy: 0.0261
Test set
  Loss: 18964541440.000
  Accuracy: 0.026


In [21]:
len(df.themes[0])

112